<a href="https://colab.research.google.com/github/srini11govind/skills-github-pages/blob/main/milestone2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re
from nltk.corpus import stopwords
from collections import Counter
import nltk

In [ ]:
df = pd.read_csv("/content/train.csv")
print("Dataset Shape:", df.shape)
df.head()

Dataset Shape: (6827, 8)


,id,text,anger,fear,joy,sadness,surprise,emotions
0,0,the dentist that did the work apparently did a...,1,0,0,1,0,['anger' 'sadness']
1,1,i'm gonna absolutely ~~suck~~ be terrible duri...,0,1,0,1,0,['fear' 'sadness']
2,2,"bridge: so leave me drowning calling houston, ...",0,1,0,1,0,['fear' 'sadness']
3,3,after that mess i went to see my now ex-girlfr...,1,1,0,1,0,['anger' 'fear' 'sadness']
4,4,"as he stumbled i ran off, afraid it might some...",0,1,0,0,0,['fear']


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def clean_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

In [ ]:
original_char_count = df['text'].apply(len).sum()
df['clean_text'] = df['text'].apply(clean_text)
clean_char_count = df['clean_text'].apply(len).sum()
char_reduction_pct = ((original_char_count - clean_char_count) / original_char_count) * 100
print(f"Percentage reduction in total character count after removing punctuation: {char_reduction_pct:.2f}%")

Percentage reduction in total character count after removing punctuation: 3.26%


In [ ]:
stop_words = set(stopwords.words('english'))
all_words = " ".join(df['clean_text']).split()
unique_words = set(all_words)
stopword_in_unique = [w for w in unique_words if w in stop_words]
stopword_percentage = (len(stopword_in_unique) / len(unique_words)) * 100
print(f"Percentage of unique words that are stop words: {stopword_percentage:.2f}%")

Percentage of unique words that are stop words: 1.54%


In [ ]:

stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))

    words = [word for word in text.split() if word not in stop_words]
    return ' '.join(words)

df['text_clean'] = df['text'].apply(clean_text)

**Convert Text to Features (BoW, TF-IDF)**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Bag-of-Words
bow = CountVectorizer()
X_bow = bow.fit_transform(df['text_clean'])

# TF-IDF
tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(df['text_clean'])

** Prepare Targets and Train-Test Split**

In [ ]:
from sklearn.model_selection import train_test_split


labels = ["anger", "fear", "joy", "sadness", "surprise"]
y = df[labels]

# Split
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

**Train Classifier (Logistic Regression example)**

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

model = OneVsRestClassifier(LogisticRegression(max_iter=200))
model.fit(X_train, y_train)

OneVsRestClassifier(estimator=LogisticRegression(max_iter=200))

**Evaluate using Macro F1**

In [ ]:
from sklearn.metrics import f1_score

y_pred = model.predict(X_test)
f1 = f1_score(y_test, y_pred, average='macro')
print(f"Macro F1 Score: {f1:.4f}")


Macro F1 Score: 0.4063


In [ ]:
from xgboost import XGBClassifier

model_xgb = OneVsRestClassifier(XGBClassifier(use_label_encoder=False, eval_metric="logloss"))
model_xgb.fit(X_train, y_train)

/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [07:07:18] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [07:07:19] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [07:07:20] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [07:07:21] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [07:07:23] WARNING: /w

OneVsRestClassifier(estimator=XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric='logloss',
                                            feature_types=None,
                                            feature_weights=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=None, n_jobs=None,
                                            num_parallel_tree=None, ...))

In [ ]:
from sklearn.metrics import f1_score

y_pred1 = model_xgb.predict(X_test)
f12 = f1_score(y_test, y_pred1, average='macro')
print(f"Macro F1 Score: {f1:.4f}")

Macro F1 Score: 0.4063


In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score

nb_model = OneVsRestClassifier(MultinomialNB())
nb_model.fit(X_train, y_train)
y_pred_nb = nb_model.predict(X_test)
f1_nb = f1_score(y_test, y_pred_nb, average='macro')
print(f"Naive Bayes Macro F1: {f1_nb:.4f}")

Naive Bayes Macro F1: 0.4119


In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 10.0 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score

catboost_wrapped = OneVsRestClassifier(
    CatBoostClassifier(
        iterations=500,
        learning_rate=0.1,
        loss_function='Logloss',  # Binary classification loss
        eval_metric='F1',
        verbose=100
    )
)

catboost_wrapped.fit(X_train.toarray(), y_train)
y_pred_cat = catboost_wrapped.predict(X_test.toarray())
f1_cat = f1_score(y_test, y_pred_cat, average='macro')
print(f"CatBoost Macro F1: {f1_cat:.4f}")

0:	learn: 0.0030864	total: 89.1ms	remaining: 44.5s
100:	learn: 0.1184971	total: 3.69s	remaining: 14.6s
200:	learn: 0.2319236	total: 8.2s	remaining: 12.2s
300:	learn: 0.3900621	total: 11.8s	remaining: 7.8s
400:	learn: 0.4657210	total: 15.4s	remaining: 3.81s
499:	learn: 0.6514048	total: 19.9s	remaining: 0us
0:	learn: 0.7205371	total: 36.3ms	remaining: 18.1s
100:	learn: 0.7886952	total: 3.72s	remaining: 14.7s
200:	learn: 0.8539683	total: 7.32s	remaining: 10.9s
300:	learn: 0.8912816	total: 11.9s	remaining: 7.88s
400:	learn: 0.9183116	total: 15.5s	remaining: 3.84s
499:	learn: 0.9357516	total: 19.1s	remaining: 0us
0:	learn: 0.0029806	total: 40.8ms	remaining: 20.4s
100:	learn: 0.3105823	total: 4.58s	remaining: 18.1s
200:	learn: 0.4991643	total: 8.16s	remaining: 12.1s
300:	learn: 0.6015584	total: 11.8s	remaining: 7.82s
400:	learn: 0.6945119	total: 16.3s	remaining: 4.04s
499:	learn: 0.7535963	total: 19.9s	remaining: 0us
0:	learn: 0.0360563	total: 35.4ms	remaining: 17.6s
100:	learn: 0.3935164	to

In [ ]:
import lightgbm as lgb

lgbm_model = OneVsRestClassifier(
    lgb.LGBMClassifier(n_estimators=100, random_state=42)
)
lgbm_model.fit(X_train, y_train)
y_pred_lgb = lgbm_model.predict(X_test)
f1_lgb = f1_score(y_test, y_pred_lgb, average='macro')
print(f"LightGBM Macro F1: {f1_lgb:.4f}")

[LightGBM] [Info] Number of positive: 647, number of negative: 4814
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.018802 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6821
[LightGBM] [Info] Number of data points in the train set: 5461, number of used features: 394
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.118476 -> initscore=-2.006937
[LightGBM] [Info] Start training from score -2.006937
[LightGBM] [Info] Number of positive: 3102, number of negative: 2359
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.019472 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6821
[LightGBM] [Info] Number of data points in the train set: 5461, number of used features: 394
[LightGBM] [Info] [bina

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [ ]:
import numpy as np

# Per-label F1 score
per_label_f1 = f1_score(y_test, y_pred_lgb, average=None)
for idx, label in enumerate(labels):
    print(f"{label}: F1={per_label_f1[idx]:.3f}")

# Rows where at least one label is wrongly predicted
incorrect = np.where((y_test != y_pred_lgb).any(axis=1))[0]
print(f"Total incorrect predictions: {len(incorrect)}")

# Show first few misclassified examples
misclassified_df = df.iloc[y_test.index[incorrect]]
print(misclassified_df[['text'] + labels].head(3))

anger: F1=0.212
fear: F1=0.719
joy: F1=0.353
sadness: F1=0.499
surprise: F1=0.495
Total incorrect predictions: 985
                                                   text  anger  fear  joy  \
3916  it took a while, my elbows were stiff, like no...      0     1    0   
1174  after a bus ride, i found myself in a small to...      0     1    0   
712                      y'all are so awesome actually!      0     0    1   

      sadness  surprise  
3916        1         0  
1174        0         0  
712         0         0  


In [ ]:
tfidf_full = TfidfVectorizer(ngram_range=(1, 2), stop_words='english')
tfidf_full.fit(df['text'])
print("Total unique tokens without limit:", len(tfidf_full.get_feature_names_out()))

Total unique tokens without limit: 33896


Q1.Utilise the TfidfVectorizer to tokenize the text feature dataset, setting the following parameters: ngram_range=(1, 2) and stop_words=‘english’.

Subsequently, provide answers to the following questions based on the tokenized data:   ***What is the dimension of your feature matrix?***


In [ ]:
tfidf = TfidfVectorizer(

    ngram_range=(1, 2),
    stop_words='english'
)

X_tfidf = tfidf.fit_transform(df['text'])

**Ans1.**

In [ ]:
print("TF-IDF feature matrix shape:", X_tfidf.shape)

TF-IDF feature matrix shape: (6827, 33896)


In [ ]:
non_zero = X_tfidf.nnz
total = X_tfidf.shape[0] * X_tfidf.shape[1]
sparsity = 100 * (1 - (non_zero / total))
print(f"Sparsity of TF-IDF matrix: {sparsity:.2f}%")

Sparsity of TF-IDF matrix: 99.96%


In [ ]:
nonzero_per_doc = X_tfidf.getnnz(axis=1)
print(f"Average number of non-zero tokens per document: {nonzero_per_doc.mean():.2f}")

Average number of non-zero tokens per document: 12.28


**Each row or document uses only 13 to 14 tokens in avg. Maximum of  0.04% tokens only required per row on average**

**Q2. Is this feature representation sparse or dense?**

In [ ]:
type(X_tfidf)

scipy.sparse._csr.csr_matrix

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score

X_train_text, X_test_text, y_train, y_test = train_test_split(
    df['text'], y, test_size=0.2, random_state=42
)

tfidf = TfidfVectorizer(ngram_range=(1,2), stop_words='english')
X_train = tfidf.fit_transform(X_train_text)
X_test  = tfidf.transform(X_test_text)
clf = DecisionTreeClassifier(random_state=42, max_depth=6)
clf.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=6, random_state=42)

In [ ]:
n_nodes = clf.tree_.node_count
print("Number of nodes in the Decision Tree:", n_nodes)

Number of nodes in the Decision Tree: 55


In [ ]:
y_pred_train = clf.predict(X_train)
y_pred_test = clf.predict(X_test)
f1_train = f1_score(y_train, y_pred_train, average='macro')
f1_test = f1_score(y_test, y_pred_test, average='macro')

print(f"Macro F1 Score (Training): {f1_train:.4f}")
print(f"Macro F1 Score (Test): {f1_test:.4f}")

Macro F1 Score (Training): 0.2146
Macro F1 Score (Test): 0.1972
